In [1]:
#import required files
!git clone https://github.com/RishiNandha/AWS_Dataset

Cloning into 'AWS_Dataset'...
remote: Enumerating objects: 3945, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 3945 (delta 0), reused 4 (delta 0), pack-reused 3941
Receiving objects: 100% (3945/3945), 70.17 MiB | 27.89 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (5654/5654), done.


In [3]:
import os
os.chdir('AWS_Dataset')

In [52]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model


In [37]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split = 0.2,
                                   fill_mode='nearest')

In [38]:
# Flow training images in batches using train_datagen
train_generator = train_datagen.flow_from_directory(
    '/content/AWS_Dataset/Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Use subset='training' for training data
)

# Flow validation images in batches using train_datagen
validation_generator = train_datagen.flow_from_directory(
    '/content/AWS_Dataset/Dataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Use subset='validation' for validation data
)


Found 4514 images belonging to 6 classes.
Found 1126 images belonging to 6 classes.


In [45]:
base_model = MobileNet(input_shape=(224,224,3),
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classifier_activation="softmax")

17225924/17225924 [==============================] - 1s 0us/step


In [46]:
base_model.trainable = False

In [47]:
num_classes = 6
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation = 'relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [48]:
model = Model(inputs=base_model.input, outputs=predictions)

In [49]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
checkpoint_path = "checkpoint.h5"
batch_size = 32
epochs = 20
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train your model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs= epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[checkpoint])

# Load the best checkpoint for final evaluation
model.load_weights(checkpoint_path)

Epoch 1/20
141/141 [==============================] - ETA: 0s - loss: 0.4591 - accuracy: 0.8340
Epoch 1: val_accuracy improved from -inf to 0.83125, saving model to checkpoint.h5
141/141 [==============================] - 76s 541ms/step - loss: 0.4591 - accuracy: 0.8340 - val_loss: 0.4989 - val_accuracy: 0.8313


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/20
141/141 [==============================] - ETA: 0s - loss: 0.3650 - accuracy: 0.8693
Epoch 2: val_accuracy improved from 0.83125 to 0.86607, saving model to checkpoint.h5
141/141 [==============================] - 77s 544ms/step - loss: 0.3650 - accuracy: 0.8693 - val_loss: 0.4357 - val_accuracy: 0.8661
Epoch 3/20
141/141 [==============================] - ETA: 0s - loss: 0.3136 - accuracy: 0.8911
Epoch 3: val_accuracy did not improve from 0.86607
141/141 [==============================] - 76s 545ms/step - loss: 0.3136 - accuracy: 0.8911 - val_loss: 0.4192 - val_accuracy: 0.8500
Epoch 4/20
141/141 [==============================] - ETA: 0s - loss: 0.2672 - accuracy: 0.9105
Epoch 4: val_accuracy improved from 0.86607 to 0.86875, saving model to checkpoint.h5
141/141 [==============================] - 72s 512ms/step - loss: 0.2672 - accuracy: 0.9105 - val_loss: 0.4117 - val_accuracy: 0.8687
Epoch 5/20
141/141 [==============================] - ETA: 0s - loss: 0.2412 - accuracy

In [58]:
model = load_model('checkpoint.h5')
checkpoint = ModelCheckpoint('checkpoint.h5', monitor='val_accuracy', save_best_only=True)
# Continue training with additional epochs
additional_epochs = 10
history = model.fit(train_generator, epochs=additional_epochs, validation_data=validation_generator, callbacks=[checkpoint])

Epoch 1/10
142/142 [==============================] - 74s 511ms/step - loss: 0.0736 - accuracy: 0.9730 - val_loss: 0.4590 - val_accuracy: 0.8979
Epoch 2/10
142/142 [==============================] - 73s 513ms/step - loss: 0.1020 - accuracy: 0.9646 - val_loss: 0.3731 - val_accuracy: 0.9023
Epoch 3/10
142/142 [==============================] - 74s 522ms/step - loss: 0.0844 - accuracy: 0.9734 - val_loss: 0.2988 - val_accuracy: 0.9281
Epoch 4/10
142/142 [==============================] - 73s 515ms/step - loss: 0.0571 - accuracy: 0.9829 - val_loss: 0.3333 - val_accuracy: 0.9210
Epoch 5/10
142/142 [==============================] - 72s 507ms/step - loss: 0.0541 - accuracy: 0.9816 - val_loss: 0.4842 - val_accuracy: 0.9032
Epoch 6/10
142/142 [==============================] - 69s 490ms/step - loss: 0.0537 - accuracy: 0.9814 - val_loss: 0.3489 - val_accuracy: 0.9263
Epoch 7/10
142/142 [==============================] - 71s 501ms/step - loss: 0.0496 - accuracy: 0.9821 - val_loss: 0.2918 - val_ac

In [59]:
model.load_weights(checkpoint_path)

In [60]:
train_accuracy = model.evaluate(train_generator, steps=train_generator.samples // batch_size)[1]
print("Train Accuracy:", train_accuracy)

141/141 [==============================] - 57s 402ms/step - loss: 0.0396 - accuracy: 0.9883
Train Accuracy: 0.9882535338401794


In [61]:
validation_accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // batch_size)[1]
print("Validation Accuracy:", validation_accuracy)

35/35 [==============================] - 14s 396ms/step - loss: 0.3090 - accuracy: 0.9277
Validation Accuracy: 0.9276785850524902
